In [1]:
import numpy as np
import pandas as pd

In [2]:
od_agg = pd.read_csv('od_aggregated.csv')
od_agg['2percentflow'] = round(od_agg['car'] * 0.02)
od_agg

,STB,STB_D,foot,bike,pt,car,all,2percentflow
0,10,10,34.636186,4.670167,0.000000,0.000000,39.306353,0.0
1,10,101,0.499550,1.608387,4.019999,9.504735,15.632671,0.0
2,10,102,0.032618,0.125923,0.439532,0.702992,1.301065,0.0
3,10,103,0.027549,0.117798,0.322680,0.719588,1.187615,0.0
4,10,104,0.001368,0.042433,0.096354,0.199188,0.339343,0.0
...,...,...,...,...,...,...,...,...
32756,86,82,0.187764,0.264061,0.023572,0.282512,0.757909,0.0
32757,86,83,0.035190,0.040958,0.008689,0.041571,0.126408,0.0
32758,86,84,2.072918,0.863023,0.023705,0.269327,3.228973,0.0
32759,86,85,0.060818,0.043419,0.000951,0.047946,0.153134,0.0


In [ ]:
od_count = od_agg.groupby(['STB'])['2percentflow'].sum()
od_count = pd.DataFrame({'STB':od_count.index, 'count':od_count.values})
od_count['count'] = od_count['count'].astype('int')
od_count

In [ ]:
ratio_day = pd.read_csv('ratio_weekday.csv')

origin_hourly_dist = pd.DataFrame()
for i in range(len(od_count)):
    col = []
    for j in range(len(ratio_day)):
        col.append(round(od_count['count'].iloc[i] * ratio_day['ratio'].iloc[j]))
    origin_hourly_dist[od_count['STB'].iloc[i]] = col
    
origin_hourly_dist = origin_hourly_dist.loc[:, (origin_hourly_dist != 0).any(axis=0)]
origin_hourly_dist

In [3]:
stops_final = pd.read_csv('stops_table_final.csv')
stops_final.drop(stops_final.columns[[-1,]], axis=1, inplace=True)
stops_final.drop(stops_final.tail(12).index, inplace = True)
stops_final[['StopID','RegionID']] = stops_final[['StopID','RegionID']].astype('int')
stops_final

,StopID,RegionID,stop_lat,stop_lon
0,1,302,52.118363,11.606746
1,2,243,52.123984,11.606198
2,3,243,52.127365,11.605738
3,4,263,52.129757,11.605294
4,5,244,52.130259,11.611049
...,...,...,...,...
117,118,383,52.089195,11.607802
118,119,702,52.194149,11.673130
119,120,702,52.202900,11.674533
120,121,702,52.204192,11.673836


In [4]:
import random
def decomposition(i, j):
    a =[]
    for k in range(j-1):
        n = random.randint(1, i)
        a.append(n)
        i -= n
        if i<=0:
            break
    if i > 0:
        a.append(i)
    return a

In [16]:
from math import radians, cos, sin, asin, sqrt
def find_dist(lat1, long1, lat2, long2):
    # convert decimal degrees to radians 
    lat1, long1, lat2, long2 = map(radians, [lat1, long1, lat2, long2])
    # haversine formula 
    dlon = long2 - long1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    # Radius of earth in kilometers is 6371
    km = 6371* c
    return km

In [17]:
def find_nearest(lat, long):
    distances = stops_final.apply(
        lambda row: find_dist(lat, long, row['stop_lat'], row['stop_lon']), 
        axis=1)
    return stops_final.loc[distances.idxmin(), 'StopID']

In [7]:
centroids_latlon = pd.read_csv('centroids_latlon.csv')


In [ ]:
centroids_latlon = pd.read_csv('centroids_latlon.csv')
trip_stop = pd.read_csv('trip_stop.csv')
outerlist = []
for regionID, demand in origin_hourly_dist.items():
    if regionID in stops_final['RegionID'].unique():
        
        if stops_final.RegionID.value_counts()[regionID] == 1:
            #print("one stop")
            stopID = [stops_final.loc[stops_final['RegionID']==regionID, 'StopID'].iloc[0]]
            #print(stopID)
            
            
        else:
            #print("too many stops")
            stopID = stops_final.loc[stops_final['RegionID']==regionID, 'StopID'].tolist()
            #print(stopID)
    else:
        #print('no stops')
        lat = centroids_latlon.loc[centroids_latlon['regionID']==regionID,'latitude'].iloc[0]
        lon = centroids_latlon.loc[centroids_latlon['regionID']==regionID,'longitude'].iloc[0]
        stopID = [find_nearest(lat, lon)]
        #print(stopID)
        
    for hour in range(len(demand)):
        df = trip_stop[trip_stop['arrival_time']//60==hour]
            #print(df)
        if demand[hour] > 0:
            rand_dmd = decomposition(demand[hour],len(stopID))
            for s in range(len(rand_dmd)):
                df2 = df[df['StopID']==stopID[s]]
                if len(df2) == 0:
                    lat = centroids_latlon.loc[centroids_latlon['regionID']==regionID,'latitude'].iloc[0]
                    lon = centroids_latlon.loc[centroids_latlon['regionID']==regionID,'longitude'].iloc[0]
                    a = find_nearest(lat, lon)
                    df = trip_stop[trip_stop['StopID']==a]
                    df2 = df[df['arrival_time']//60==hour]
                #print(demand[hour], len(df2))
                #for d in rand_dmd:
                random_demand = decomposition(rand_dmd[s],len(df2))
                for i in random_demand:
                    if len(df2) > 0:
                        row = df2.sample()
                        df2.drop(row.index)
                        innerlist = [stopID[s], int(row['arrival_time']), i, regionID]
                        outerlist.append(innerlist)
                            
origin_demands = pd.DataFrame(outerlist)
origin_demands.columns = ['StopID', 'ArrivalTime', 'NumberCustomers', 'OriginRegionID']
origin_demands.sort_values('ArrivalTime').to_excel('origin_demands.xlsx', index=False)

In [8]:
od = pd.read_excel('origin_demands.xlsx')
od = od.sort_values('OriginRegionID').reset_index(drop=True)

In [9]:
od_agg = od_agg.astype({"STB": int})
od_agg = od_agg.sort_values('STB').reset_index(drop=True)

In [10]:
od_agg

,STB,STB_D,foot,bike,pt,car,all,2percentflow
0,10,10,34.636186,4.670167,0.000000,0.000000,39.306353,0.0
1,10,482,0.004920,0.140272,1.607944,3.117859,4.870995,0.0
2,10,483,0.002700,0.340302,0.783563,1.037576,2.164141,0.0
3,10,484,0.001397,0.050708,0.512220,2.215143,2.779468,0.0
4,10,485,0.000082,0.011329,0.043254,0.049916,0.104581,0.0
...,...,...,...,...,...,...,...,...
32756,784,301,0.090694,0.630959,1.358704,4.063084,6.143441,0.0
32757,784,302,0.094649,0.574677,1.445176,6.581734,8.696236,0.0
32758,784,303,0.133421,1.275312,2.204112,15.377271,18.990116,0.0
32759,784,305,0.069552,0.713543,2.829289,13.281437,16.893821,0.0


In [11]:
index_df1 = 0
index_df2 = 0

while index_df1 < len(od) and index_df2 < len(od_agg):
    value_od = od.loc[index_df1, 'OriginRegionID']
    value_odagg = od_agg.loc[index_df2, 'STB']
    
    if value_od == value_odagg:
        dem = od_agg.loc[index_df2, '2percentflow']
        if dem > 0:
            od.loc[index_df1, 'DestinationRegionID'] = od_agg.loc[index_df2, 'STB_D']
            od.loc[index_df1, 'Demand'] = dem
            index_df1 += 1
            index_df2 += 1
        else:
            index_df2 += 1

    elif value_od > value_odagg:
        index_df2 += 1
    else:
        index_df1 += 1
        

print(od)

      StopID  ArrivalTime  NumberCustomers  OriginRegionID  \
0         10          790                2              11   
1         10          594                1              11   
2         10         1020                3              11   
3         10         1098                8              11   
4         11         1306                4              11   
...      ...          ...              ...             ...   
1493      62         1048                1             784   
1494      62          538                1             784   
1495      62         1098                1             784   
1496      62         1171                1             784   
1497      62          986                1             784   

      DestinationRegionID  Demand  
0                   501.0     1.0  
1                   482.0     2.0  
2                   484.0     1.0  
3                   502.0     1.0  
4                   543.0     1.0  
...                   ...     ...  
149

In [12]:
trip_stop = pd.read_csv('trip_stop.csv')

In [22]:
od = od.fillna(0)
od = od.astype({"DestinationRegionID": int})
outerlist = []
for i in range(len(od)):
    regionID = od['DestinationRegionID'].iloc[i]
    if regionID in stops_final['RegionID'].unique():
        
        if stops_final.RegionID.value_counts()[regionID] == 1:
            stopID = [stops_final.loc[stops_final['RegionID']==regionID, 'StopID'].iloc[0]]
        else:
            stopID = stops_final.loc[stops_final['RegionID']==regionID, 'StopID'].tolist()
    elif regionID > 0:
        lat = centroids_latlon.loc[centroids_latlon['regionID']==regionID,'latitude'].iloc[0]
        lon = centroids_latlon.loc[centroids_latlon['regionID']==regionID,'longitude'].iloc[0]
        stopID = [find_nearest(lat, lon)]
    
    desti = random.choice(stopID)
    hr = od['ArrivalTime'].iloc[i]//60
    distance= pd.read_excel("distODh"+str(hr)+".xlsx", index_col = 0)
    time = distance.loc[od['StopID'].iloc[i], desti]
    df = trip_stop[trip_stop['arrival_time'] > od['ArrivalTime'].iloc[i]+time]
    df = df[df['StopID']==desti].sort_values('arrival_time')
    if not df.empty:
        arrival = df['arrival_time'].iloc[0]
        innerlist = [desti, arrival, od['NumberCustomers'].iloc[i], regionID]
        outerlist.append(innerlist)
    
destination_demands = pd.DataFrame(outerlist)
destination_demands.columns = ['StopID', 'ArrivalTime', 'NumberCustomers', 'DestinationRegionID']
destination_demands.sort_values('ArrivalTime').to_excel('destination_demands.xlsx', index=False)